# V0.1

En esta version se va a proiceder a dummyficar todas als variables posibles y aplicar un modelo sencillo con el que partir como baseline.

Kaggle - 0.740
AUC_X_test = 0.7424

In [66]:
import os
import pandas as pd
import numpy as np
import gc

import seaborn as sns
import matplotlib.pyplot as plt

from pandas.api.types import CategoricalDtype 

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve

from xgboost import XGBRegressor, plot_importance

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, BatchNormalization

from datetime import datetime, date

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path_input = './input/'
path_output = './output/'

In [3]:
print(os.listdir(path_input))

['sample_submission.csv', 'test.csv', 'train.csv']


In [4]:
train = pd.read_csv(path_input + 'train.csv')
test = pd.read_csv(path_input + 'test.csv')
submission = pd.read_csv(path_input + 'sample_submission.csv')

In [5]:
train.drop(['id'], inplace = True, axis = 1)
test.drop(['id'], inplace = True, axis = 1)

In [6]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [7]:
train.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2',
       'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0',
       'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month', 'target'],
      dtype='object')

In [8]:
 train.ord_3.unique()

array(['h', 'a', 'i', 'j', 'g', 'e', 'd', 'b', 'k', 'f', 'l', 'n', 'o',
       'c', 'm'], dtype=object)

In [9]:
 train.ord_0.unique()

array([2, 1, 3], dtype=int64)

## Binary Features

In [10]:
bin_dict = {'T':1, 'F':0, 'Y':1, 'N':0}

train['bin_3'] = train['bin_3'].map(bin_dict)
train['bin_4'] = train['bin_4'].map(bin_dict)

test['bin_3'] = test['bin_3'].map(bin_dict)
test['bin_4'] = test['bin_4'].map(bin_dict)

In [11]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


## Nominal Features 

In [12]:
test['target'] = 'test'
df = pd.concat([train, test], axis=0, sort=False )

print(f'Shape before dummy transformation: {df.shape}')
df = pd.get_dummies(df, columns=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'],\
                          prefix=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'], drop_first=True)
print(f'Shape after dummy transformation: {df.shape}')


Shape before dummy transformation: (500000, 24)
Shape after dummy transformation: (500000, 39)


In [13]:
train, test = df[df['target'] != 'test'], df[df['target'] == 'test'].drop('target', axis=1)
del df
gc.collect()

76

In [14]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,...,nom_2_Lion,nom_2_Snake,nom_3_China,nom_3_Costa Rica,nom_3_Finland,nom_3_India,nom_3_Russia,nom_4_Oboe,nom_4_Piano,nom_4_Theremin
0,0,0,0,1,1,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,...,0,1,0,0,1,0,0,0,0,0
1,0,1,0,1,1,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,...,0,0,0,0,0,0,1,0,1,0
2,0,0,0,0,1,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,...,1,0,0,0,0,0,1,0,0,1
3,0,1,0,0,1,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,...,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,...,1,0,0,0,0,0,0,1,0,0


## Ordinal Features

In [15]:
# seting the orders of our ordinal features
ord_1 = CategoricalDtype(categories=['Novice', 'Contributor','Expert', 
                                     'Master', 'Grandmaster'], ordered=True)
ord_2 = CategoricalDtype(categories=['Freezing', 'Cold', 'Warm', 'Hot',
                                     'Boiling Hot', 'Lava Hot'], ordered=True)
ord_3 = CategoricalDtype(categories=['a', 'b', 'c', 'd', 'e', 'f', 'g',
                                     'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o'], ordered=True)
ord_4 = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
                                     'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
                                     'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ordered=True)

In [16]:
# Transforming ordinal Features
train.ord_1 = train.ord_1.astype(ord_1)
train.ord_2 = train.ord_2.astype(ord_2)
train.ord_3 = train.ord_3.astype(ord_3)
train.ord_4 = train.ord_4.astype(ord_4)

# test dataset
test.ord_1 = test.ord_1.astype(ord_1)
test.ord_2 = test.ord_2.astype(ord_2)
test.ord_3 = test.ord_3.astype(ord_3)
test.ord_4 = test.ord_4.astype(ord_4)

In [17]:
train.ord_4.head()

0    D
1    A
2    R
3    D
4    R
Name: ord_4, dtype: category
Categories (26, object): [A < B < C < D ... W < X < Y < Z]

In [18]:
# Geting the codes of ordinal categoy's - train
train.ord_1 = train.ord_1.cat.codes
train.ord_2 = train.ord_2.cat.codes
train.ord_3 = train.ord_3.cat.codes
train.ord_4 = train.ord_4.cat.codes

# Geting the codes of ordinal categoy's - test
test.ord_1 = test.ord_1.cat.codes
test.ord_2 = test.ord_2.cat.codes
test.ord_3 = test.ord_3.cat.codes
test.ord_4 = test.ord_4.cat.codes

In [19]:
train[['ord_0', 'ord_1', 'ord_2', 'ord_3']].head()

,ord_0,ord_1,ord_2,ord_3
0,2,4,1,7
1,1,4,3,0
2,1,2,5,7
3,1,4,4,8
4,1,4,0,0


## Date Features

In [20]:
train['day'].head()

0    2
1    7
2    7
3    2
4    7
Name: day, dtype: int64

In [21]:
date_cols = ['day', 'month']

def date_cyc_enc(df, col, max_vals):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
    return df

train = date_cyc_enc(train, 'day', 7)
test = date_cyc_enc(test, 'day', 7) 

train = date_cyc_enc(train, 'month', 12)
test = date_cyc_enc(test, 'month', 12)

In [22]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,...,nom_3_Finland,nom_3_India,nom_3_Russia,nom_4_Oboe,nom_4_Piano,nom_4_Theremin,day_sin,day_cos,month_sin,month_cos
0,0,0,0,1,1,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,...,1,0,0,0,0,0,9.749279e-01,-0.222521,0.866025,0.500000
1,0,1,0,1,1,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,...,0,0,1,0,1,0,-2.449294e-16,1.000000,-0.866025,-0.500000
2,0,0,0,0,1,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,...,0,0,1,0,0,1,-2.449294e-16,1.000000,0.866025,0.500000
3,0,1,0,0,1,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,...,0,0,0,1,0,0,9.749279e-01,-0.222521,0.500000,0.866025
4,0,0,0,0,0,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,...,0,0,0,1,0,0,-2.449294e-16,1.000000,-0.866025,-0.500000


## Hashing Features

In [23]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,...,nom_3_Finland,nom_3_India,nom_3_Russia,nom_4_Oboe,nom_4_Piano,nom_4_Theremin,day_sin,day_cos,month_sin,month_cos
0,0,0,0,1,1,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,...,1,0,0,0,0,0,9.749279e-01,-0.222521,0.866025,0.500000
1,0,1,0,1,1,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,...,0,0,1,0,1,0,-2.449294e-16,1.000000,-0.866025,-0.500000
2,0,0,0,0,1,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,...,0,0,1,0,0,1,-2.449294e-16,1.000000,0.866025,0.500000
3,0,1,0,0,1,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,...,0,0,0,1,0,0,9.749279e-01,-0.222521,0.500000,0.866025
4,0,0,0,0,0,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,...,0,0,0,1,0,0,-2.449294e-16,1.000000,-0.866025,-0.500000


In [24]:
hash_cols = ['ord_5', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
test['target'] = 'test'
df = pd.concat([train, test], axis=0, sort=False )

tfidf_vect = TfidfVectorizer()

for col in hash_cols:
    x = tfidf_vect.fit_transform(df[col])
    df[col] = np.asarray(x.mean(axis=1)).ravel().tolist()

In [25]:
train, test = df[df['target'] != 'test'], df[df['target'] == 'test'].drop('target', axis=1)

In [26]:
print(train.shape)
print(test.shape)

(300000, 43)
(200000, 42)


In [27]:
train.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_5', 'nom_6', 'nom_7',
       'nom_8', 'nom_9', 'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5',
       'day', 'month', 'target', 'nom_0_Green', 'nom_0_Red', 'nom_1_Polygon',
       'nom_1_Square', 'nom_1_Star', 'nom_1_Trapezoid', 'nom_1_Triangle',
       'nom_2_Cat', 'nom_2_Dog', 'nom_2_Hamster', 'nom_2_Lion', 'nom_2_Snake',
       'nom_3_China', 'nom_3_Costa Rica', 'nom_3_Finland', 'nom_3_India',
       'nom_3_Russia', 'nom_4_Oboe', 'nom_4_Piano', 'nom_4_Theremin',
       'day_sin', 'day_cos', 'month_sin', 'month_cos'],
      dtype='object')

## Modelling

In [28]:
y = train['target']
train.drop(['target'], inplace = True, axis = 1)

C:\Users\Enric\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [71]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.1, random_state = 12)

In [72]:
my_model = Sequential()
my_model.add(Dense(64, activation = 'relu', input_dim = len(test.columns)))
my_model.add(Dropout(0.3))
my_model.add(BatchNormalization())
my_model.add(Dense(32, activation = 'relu'))
my_model.add(Dropout(0.4))
my_model.add(BatchNormalization())
my_model.add(Dense(16, activation = 'relu'))
my_model.add(Dropout(0.2))
my_model.add(BatchNormalization())
my_model.add(Dense(1, activation = 'sigmoid'))

my_model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 64)                2752      
_________________________________________________________________
dropout_23 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
dense_30 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_24 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 32)                128       
_________________________________________________________________
dense_31 (Dense)             (None, 16)                528       
__________

In [73]:
my_model.fit(X_train, y_train,
             validation_data = (X_test, y_test),
             batch_size = 128, epochs = , verbose = 1)

Train on 270000 samples, validate on 30000 samples
Epoch 1/7
270000/270000 [==============================] - 6s 21us/step - loss: 0.5744 - acc: 0.7018 - val_loss: 0.5410 - val_acc: 0.7278
Epoch 2/7
270000/270000 [==============================] - 5s 18us/step - loss: 0.5476 - acc: 0.7212 - val_loss: 0.5374 - val_acc: 0.7323
Epoch 3/7
270000/270000 [==============================] - 5s 17us/step - loss: 0.5441 - acc: 0.7247 - val_loss: 0.5359 - val_acc: 0.7325
Epoch 4/7
270000/270000 [==============================] - 5s 19us/step - loss: 0.5425 - acc: 0.7256 - val_loss: 0.5371 - val_acc: 0.7321
Epoch 5/7
270000/270000 [==============================] - 5s 18us/step - loss: 0.5424 - acc: 0.7260 - val_loss: 0.5401 - val_acc: 0.7318
Epoch 6/7
270000/270000 [==============================] - 5s 17us/step - loss: 0.5418 - acc: 0.7264 - val_loss: 0.5367 - val_acc: 0.7294
Epoch 7/7
270000/270000 [==============================] - 5s 17us/step - loss: 0.5416 - acc: 0.7265 - val_loss: 0.5368 -

In [74]:
y_pred = my_model.predict(X_test, verbose = 1)

30000/30000 [==============================] - 1s 20us/step


In [75]:
roc_auc_score(np.array(y_test).astype(int), y_pred.reshape(y_pred.shape[0]))

0.7394385722441854

## Submission

In [43]:
y_preds = my_model.predict(test, verbose = 1)

200000/200000 [==============================] - 2s 9us/step


In [48]:
submission['target'] = y_preds
submission.to_csv(path_output + 'DL_0.csv', index = False)

In [45]:
submission.target.mean()

0.3080156743526459

In [47]:
submission.head()

,id,target
0,300000,0.287078
1,300001,0.512232
2,300002,0.171869
3,300003,0.318102
4,300004,0.530699
